In [9]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sys
sys.path.append('../experiments/')
results_dir = '../results/'
experiment_filename = '../experiments/02_distill_featurized_model.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 144/144 [00:00<00:00, 1295.34it/s]

experiment varied these params: ['model_name', 'distiller_name', 'featurizer_name', 'featurizer_frac', 'featurizer_overlap', 'depth', 'pre_interaction', 'gpu']


model_name      distiller_name  featurizer_name  featurizer_frac  featurizer_overlap  depth  pre_interaction  gpu
ft_transformer  figs            featurizer       0.33             0                   2      l0l2             0.0    1
                                                                                             l1l2             0.0    1
                                                                                      3      l0l2             0.0    1
                                                                                             l1l2             0.0    1
                                                                  1                   2      l0l2             0.0    1
                                                                                                                    ..
resnet          ft_distill      featurizer       0.75             0                   3      l1l2             0.0    1
                                                     

In [20]:
r.sort_values('teacher_r2_score_test_true', ascending=False)

,dataset_name,subsample_frac,seed,save_dir,model_name,distiller_name,featurizer_name,featurizer_frac,featurizer_overlap,depth,...,distiller_r2_score_test_true,distiller_r2_score_train_teacher,distiller_r2_score_test_teacher,teacher_f_r2_score_train_true,teacher_f_r2_score_test_true,distiller_f_r2_score_train_true,distiller_f_r2_score_test_true,distiller_f_r2_score_train_teacher,distiller_f_r2_score_test_teacher,gpu
21,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,no_featurizer,0.33,0,3,...,0.605583,0.773258,0.753633,NaN,NaN,NaN,NaN,NaN,NaN,0.0
44,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,no_featurizer,0.50,1,3,...,0.602634,0.741268,0.725159,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,no_featurizer,0.50,1,3,...,0.596428,0.739353,0.723082,NaN,NaN,NaN,NaN,NaN,NaN,0.0
103,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,no_featurizer,0.33,1,3,...,0.630801,0.777838,0.762769,NaN,NaN,NaN,NaN,NaN,NaN,0.0
118,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,no_featurizer,0.33,0,2,...,0.589612,0.749734,0.729971,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,ft_distill,featurizer,0.75,0,2,...,0.558979,0.875879,0.874036,0.614390,0.565230,0.596006,0.573853,0.922410,0.919577,0.0
29,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,ft_distill,featurizer,0.75,0,2,...,0.556231,0.871405,0.874327,0.603499,0.520525,0.597471,0.561354,0.862333,0.869247,0.0
95,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,ft_distill,featurizer,0.75,0,2,...,0.553315,0.840548,0.841509,0.601134,0.518414,0.585382,0.552844,0.839919,0.841015,0.0
67,ca_housing,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,figs,featurizer,0.75,0,2,...,0.555863,0.818943,0.818197,0.608629,0.520131,0.588562,0.555225,0.814735,0.811790,0.0


In [4]:
# group using these experiment hyperparams when averaging over random seeds
ravg = imodelsx.process_results.average_over_seeds(
    r, experiment_filename, key_to_average_over='seed'
)

# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='distiller_r2_score_test_true', ascending=False)
    .groupby(by=['model_name', 'dataset_name'])
    .max()
    .reset_index()
)
ravg_cv

,model_name,dataset_name,subsample_frac,save_dir,distiller_name,featurizer_name,featurizer_frac,featurizer_overlap,depth,bit,...,teacher_f_r2_score_test_true,teacher_f_r2_score_test_true_err,distiller_f_r2_score_train_true,distiller_f_r2_score_train_true_err,distiller_f_r2_score_test_true,distiller_f_r2_score_test_true_err,distiller_f_r2_score_train_teacher,distiller_f_r2_score_train_teacher_err,distiller_f_r2_score_test_teacher,distiller_f_r2_score_test_teacher_err
0,ft_transformer,ca_housing,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.695809,0.0,0.681597,0.0,0.658418,0.0,0.954380,0.0,0.952442,0.0
1,resnet,ca_housing,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.688477,0.0,0.695642,0.0,0.673698,0.0,0.907862,0.0,0.907403,0.0


In [5]:
# , hue='dataset_name')
sns.barplot(data=ravg_cv, x='model_name', y='accuracy_test')
plt.show()

ValueError: Could not interpret input 'accuracy_test'

Load an individual model

In [ ]:
run_args = r.iloc[0]
model = joblib.load(join(run_args.save_dir_unique, 'model.pkl'))
model